# Подключение

In [1]:
import pyodbc

In [2]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.199.13.60;DATABASE=rway;UID=vkomarnitskii;PWD=Rway1')
cursor = conn.cursor()


## Pandas

In [3]:
import pandas as pd
from math import isnan

Тут можно установить параметры отображения для pandas. Сколько показывать строк и столбцов соответственно.

In [4]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 150

### Отладка


In [27]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции
    """
    import time
    
    def wrapper(*args, **kwargs):
        t = time.time()
        res = func(*args, **kwargs)
        print(f'{func.__name__} выполнилась за время {time.time() - t:.2f}s')
        return res
    
    return wrapper

## Необходимые функции

Позволяет получить список характеристик для всех предложений из задачи `task_id`. 

In [28]:
@benchmark
def get_harks_by_object_keys(task_id):
    """
    Позволяет получить список характеристик для всех предложений из задачи task_id. 
    :param task_id: id задачи
    :type task_id: str
    :return: DataFrame, в котором указаны характеристики и их значения для всех объектов из задачи task_id
    """
    
#     in_expr = f'({", ".join(list_of_keys)})'
    query = '''
    SELECT
    t_har.Наименование,
    CASE 
        WHEN t.Значение_Тип = 04 THEN CAST(t.Значение_Дата AS varchar)
        WHEN t.Значение_Тип = 03 THEN CAST(t.Значение_Число AS varchar)
        WHEN t.Значение_Тип = 05 THEN t.Значение_Строка
        WHEN t.Значение_Тип = 01 THEN 
        CASE
            WHEN t.Значение = 0x869CEA7372E5CA3E468F18025D610C0B THEN 'Да'
            WHEN t.Значение = 0xB0CD188371B9A88E431B073454A9489F THEN 'Нет'
            ELSE 'НетДанных'
        END
        WHEN t.Значение_Тип = 08 THEN 
        COALESCE(
            (SELECT Наименование FROM [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop WHERE t_dop.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types WHERE t_types.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыСделки] t_types_1 WHERE t_types_1.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ЛогическиеЗначения] t_logic WHERE t_logic.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыВерифицированности] t_verif WHERE t_verif.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ПодСегменты] t_podseg WHERE t_podseg.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[Застройщики] t_zastr WHERE t_zastr.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations WHERE t_stations.Ссылка = t.Значение)                  
        )
    END
        AS Значение,
    t.Объект AS Ссылка
FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
    INNER JOIN [rway].[ПВХ].[Характеристики] t_har
        ON t_har.[Ссылка] = t.[Характеристика]
    INNER JOIN [rway].[РегистрСведений].[ПредложенияЗадач] reg_task
        ON reg_task.Предложение = t.Объект
        AND reg_task.Задача in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{}')
    '''.format(task_id)
    
    cursor.execute(query)
#     data_df = pd.read_sql(sql=query, con=conn)

    # Преобразование данных, которые вернул запрос в список списков, одновременно все байты преобразуются в hex
    data = list(map(lambda x: list(x), cursor.fetchall()))
    data_df = pd.DataFrame(data, columns=list(map(lambda x: x[0], cursor.description)))
    # Создание словаря для DataFrame
    return data_df.pivot(index='Ссылка', columns='Наименование', values='Значение')

    

Функция позволяет получить все предложения по конкретному заданию. Задание указывается в виде строки id задания.

`'0001-0405'`

In [7]:
@benchmark
def get_base_info_by_task_id(task_id, offers_count=None):
    """
    :param task_id: id задания
    :param offers_count: Количество предложений для выбора из базы (пустое для выбора всех предложений)
    :type task_id: str
    :return: DataFrame, содержащий все поля, которые можно получить из таблицы "ПредложенияЗадач"
    """
    
    # Запрос к дате задания
    date = cursor.execute("SELECT Дата FROM [rway].[Документ].[Задание] WHERE КодЗадания = '{}'".format(task_id)).fetchone()
    
    top = '' if not offers_count else 'TOP {}'.format(offers_count)
    query = '''
    SELECT {top}
          t.Код,
          t.Ссылка,
          t_task._Fld198 AS КодЗадачи,
          t_source.Наименование AS Источник,
          t.АдресAhunter,
          t.АктуальнаяСсылкаИсточника,
          t.ДатаПересмотраЭкспозиции,
          t.ДатаПроверкиАктуальности,
          t.ДатаРазмещения,
          t.Город,
          t.Описание,
          (SELECT Наименование FROM [rway].[Справочник].[Подсегменты] WHERE Ссылка = t.Подсегмент) AS Подсегмент,
          (SELECT Наименование FROM [rway].[Справочник].[Сегменты] WHERE Ссылка = t.Сегмент) AS Сегмент,
          (SELECT Наименование FROM [rway].[Справочник].[СубъектыРФ] WHERE Ссылка = t.Субъект) AS Субъект,
          t.СсылкаИсточника,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыРынка] WHERE Ссылка = t.ТипРынка) AS ТипРынка,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыСделки] WHERE Ссылка = t.ТипСделки) AS ТипСделки
    FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
        JOIN [rway].[РегистрСведений].[ПредложенияЗадач] t_offer
            ON Ссылка = t_offer.[Предложение]
        JOIN [rway].[dbo].[_Task62] t_task
            ON t_task.[_IDRRef] = t_offer.[Задача] AND t_task.[_IDRRef] in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{task_id}')
        JOIN [rway].[Справочник].[Источники] t_source
            ON t_source.Ссылка = ИсточникИнформации

    '''.format(top=top, task_id=task_id)
    
    cursor.execute(query)    
    data = cursor.fetchall()
    # Те же самые преобрзования, как в функции с характеристиками
    df = pd.DataFrame(map(lambda x: list(x), data))
    df.columns = list(map(lambda x: x[0], cursor.description))
    df['КодЗадания'] = task_id
    df['ДатаЗадания'] = date[0]
#     df = pd.read_sql(sql=query, con=conn)
    return df
    

# Боевой запуск

In [29]:
# Сбор основных данных из "ПредложенияОбъектовНедвижимости"
base_df = get_base_info_by_task_id('0001-0405')

get_base_info_by_task_id выполнилась за время 32.552608489990234s


In [9]:
base_df


,Код,Ссылка,КодЗадачи,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,КодЗадания,ДатаЗадания
0,190725W00299735,b'\x9aJ\xdf\x10\x9c\x9a\x1bWN\xc75\xbd\x98N]6',0001-0405-0054,MOVE.ru,"Респ Крым, г Симферополь",,2019-07-25 00:00:00,2019-07-25 16:29:56,2019-07-23 00:00:00,Симферополь,"Гипермаркет недвижимости ""ПроспектЪ"" предлагае...",офисный,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/sdaetsya_1-komnat...,Вторичный,None,0001-0405,2019-07-24 13:52:21
1,190725W00994917,b'\x9aQ\xf1@W\xb8\xf9\xd4@D\xc4\xb5\xaetlr',0001-0405-0054,MOVE.ru,"Респ Крым, г Красноперекопск, ул Калинина, дом 8",,2019-07-25 00:00:00,2019-07-25 19:27:40,2019-07-23 00:00:00,Красноперекопск,Предлогаю нежилое помещение для вашего бизнеса...,ПСН,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_psn_kr...,Вторичный,None,0001-0405,2019-07-24 13:52:21
2,190725W00976456,b'\x9axx\xd5\x8e\xff\r\xeeD\x8a\x00h\x196\xe0\...,0001-0405-0054,MOVE.ru,"Респ Крым, р-н Ленинский, с Приозерное, пер Кл...",,2019-07-25 00:00:00,2019-07-25 19:26:04,2019-07-23 00:00:00,,Продаётся торговое помещение (магазин) распол...,торговый,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_kommer...,Вторичный,None,0001-0405,2019-07-24 13:52:21
3,190725W00299718,b'\x9ay\xff\x0c&\xa9+WB\xe8\xb0 OB\xbb/',0001-0405-0054,MOVE.ru,"Респ Крым, р-н Сакский, с Лесновка, ул Гагарина",,2019-07-25 00:00:00,2019-07-25 16:29:58,2019-07-23 00:00:00,,СPОЧНАЯ ПРОДAЖА Продаeтся кoммерчecкое пoмещeн...,ПСН,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_psn_pl...,Вторичный,None,0001-0405,2019-07-24 13:52:21
4,190725W00873174,b'\x9a\x81\x07a\xa3\xe2M\xe8J\x16\x9c\xe2Rr\x84X',0001-0405-0054,MOVE.ru,"Респ Крым, г Ялта, ул Кирова, дом 51",,2019-07-25 00:00:00,2019-07-25 18:22:18,2019-07-23 00:00:00,Ялта,Сpoчнaя прoдaжа. Ялта. Цена снижeна! Kомеpческ...,ПСН,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_psn_kr...,Вторичный,None,0001-0405,2019-07-24 13:52:21
5,190725W00267139,b'\x9a\x86B\x85\xb7\xf0\x83\xf0HGd\xef\t\xbe\x...,0001-0405-0054,MOVE.ru,"Респ Крым, г Феодосия, ул Победы, дом 14Д",,2019-07-25 00:00:00,2019-07-25 16:06:28,2019-07-20 00:00:00,Феодосия,Снять помещение под коммерцию или офис в центр...,торговый,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/sdam_torgovyy_cen...,Вторичный,None,0001-0405,2019-07-24 13:52:21
6,190725W01056911,b'\x9a\x8a\x8f}\x85\x02^\xe6NJ\x19:@\x9b\xdf\x07',0001-0405-0054,MOVE.ru,"Респ Крым, г Симферополь, пр-кт Победы",,2019-07-25 00:00:00,2019-07-25 16:05:32,2019-07-23 00:00:00,Симферополь,"Гипермаркет недвижимости ""ПроспектЪ"" предлагае...",офисный,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/sdaetsya_1-komnat...,Вторичный,None,0001-0405,2019-07-24 13:52:21
7,190725W01162278,b'\x9a\xaf\x8a\x1f\x0c\xd5\xe5\xe4F\x1e\x94\xe...,0001-0405-0054,MOVE.ru,"Респ Крым, г Симферополь, пр-кт Кирова, дом 42Л",,2019-07-25 00:00:00,2019-07-25 17:49:39,2019-07-20 00:00:00,Симферополь,СИМФЕРОПОЛЬ СДАМ КАБИНЕТ В 25 кв.м. ЦЕНТРЕ Цен...,ПСН,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/sdaetsya_psn_plos...,Вторичный,None,0001-0405,2019-07-24 13:52:21
8,190725W00267454,b'\x9a\xb4\x00W\x06D\xb8\xb5B `\xe0Cb\x03\x16',0001-0405-0054,MOVE.ru,"Респ Крым, г Ялта, ул Изобильная, дом 11",,2019-07-25 00:00:00,2019-07-25 16:06:32,2019-07-24 00:00:00,Ялта,"Сдаётся офисное помещение на ул. Изобильной, ...",офисный,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/arenda_ofisnyh_po...,Вторичный,None,0001-0405,2019-07-24 13:52:21
9,190725W00509529,b'\x9a\xbc\xaa\xe7\x84\xe1q:F68\xf0^\xc4\x9b\xd7',0001-0405-0054,MOVE.ru,"Респ Крым, г Ялта, ул Маркса К., дом 18Б",,2019-07-25 00:00:00,2019-07-25 17:18:32,2019-07-23 00:00:00,Ялта,Продаётся 1/2 доля в бизнесе - две квартиры с...,о

In [10]:
# Сбор характеристик по ссылкам из основного DataFrame base_df
har_df = get_harks_by_object_keys('0001-0405')

get_harks_by_object_keys выполнилась за время 410.04589319229126s


In [11]:
har_df

Наименование,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Водоснабжение,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Интернет,Класс Объекта,Количество Комнат,Количество Этажей,Кондиционирование,Лифты,Материал Наружных Стен,Минут До Ближайшей Станции Метро,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,Охрана Предложения,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расположение Относительно Первой Линии Домов,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Объекта,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Электричество,Этаж
Ссылка,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b'\x80\x00\x01#\xc4QYuC\x82\xf6\xb5\xe4kfm',NaN,NaN,Рябикова ул,дом № 75,NaN,NaN,NaN,NaN,48.289042,54.272843,NaN,2019-07-22 00:00:00,Сдаю свободного назначения помещение 190.0 м² ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,торговая площадь,NaN,190.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Костя,кв.м.,руб./мес.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(951) 0987299,NaN,NaN,NaN,NaN,NaN,25000.000000,131.578947,NaN,1
b'\x80\x00\x02\x9c|\xb3%\x9cG\x8fmJ\x8f\xc9\xef\xd8',NaN,NaN,Космонавтов пр-кт,дом № 32В/21В,NaN,NaN,NaN,NaN,39.713863,47.296090,NaN,2019-07-25 15:57:00,"Аренда коммерческой недвижимости, 35 м2, Росто...",NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,торговая площадь,NaN,35.000000,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Продавец,кв.м.,руб./мес.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(928) 2266935,NaN,NaN,NaN,None,NaN,26000.000000,742.857143,NaN,2
b'\x80\x00\x04\x8a=+\xc1pHh)\xdc\xbbF\x9f\xe5',NaN,NaN,Рыбинская 3-я ул,"дом № 18, строение 1",NaN,NaN,NaN,NaN,37.659127,55.789683,NaN,2019-07-22 00:00:00,"Аренда коммерческой недвижимости, 286м 2, Моск...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,офис,NaN,286.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Продавец,кв.м.,руб.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(926) 9175628,NaN,NaN,NaN,NaN,NaN,572000.000000,2000.000000,NaN,"6, 2, 5"
b'\x80\x00\x05\x02\xb5\xa2\xcf\x86MG\x7fJ\xe70M&',NaN,NaN,им Бабушкина ул,,NaN,None,NaN,NaN,38.955820,45.049704,0.000000,2019-07-25 20:40:12,"Офис, 1600 м2, им Бабушкина,-",None,None,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,офис,Чистовая,1600.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Тарасова Евгения, Агентство",кв.м.,руб./мес.,NaN,0.000000,NaN,Без Проведения Торгов,NaN,NaN,(967) 9300021,NaN,бизнес-центр,NaN,бизнес-центр,NaN,183700.000000,114.812500,NaN,5
"b""\x80\x00\x06~\xb0'\xa1\x06B\xf1\x81\x91\x84Fe\x04""",NaN,NaN,Мира пр-кт,дом № 40,NaN,NaN,NaN,NaN,37.633949,55.780205,NaN,2019-07-25 21:54:35,"Помещение свободного назначения м. Пр.Мира, 27...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,офис,NaN,276.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Городской Департамент Недвижимости,кв.м.,руб./мес.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,(958) 7696931,NaN,NaN,NaN,NaN,NaN,720000.000000,2608.695652,NaN,9
b'\x80\x00\x14\xcb3\x0f\xa3QO\x01\x11r\x19\xd4\x8f\x16',NaN,NaN,Ленина пр-кт,дом № 112Б,NaN,None,NaN,0.000000,37.581900,54.154391,0.000000,2019-07-26 11:46:39,"Свободное назначение, 74 м2",NaN,НетДанных,NaN,9.000000,None,NaN,None,NaN,0.000000,Включая,свободное,None,74.000000,,Отдельный Вход,NaN,0.000000,None,74.000000,0.000000,None,NaN,NaN,Сергей Парфенов,кв.м.,руб./мес.,NaN,0.000000,None,Без Проведения Торгов,Действующий,NaN,(906) 5310114,None,NaN,None,жилой дом,,58978.000000,797.000000,NaN,
b'\x80\x00\x15\xf3\xc32#n@\xb6!F_V\xcd\xa6',NaN,NaN,Кирова пр-кт,дом № 143,NaN,None,NaN,NaN,NaN,NaN,0.000000,2019-07-25 14:13:16,"Помещение свободного назн

In [30]:
# Собираем обе таблицы в одну большую
result_df = pd.merge(base_df, har_df, on='Ссылка', how='left')

In [31]:
result_df

,Код,Ссылка,КодЗадачи,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,КодЗадания,ДатаЗадания,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Водоснабжение,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Интернет,Класс Объекта,Количество Комнат,Количество Этажей,Кондиционирование,Лифты,Материал Наружных Стен,Минут До Ближайшей Станции Метро,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,Охрана Предложения,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расположение Относительно Первой Линии Домов,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Объекта,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Электричество,Этаж
0,190725W00083177,b'\x80J\xc9^W\xa5-PE\x89\xed\xe7\xc8J\xa5\x0f',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:27:01,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,2019-07-24 13:52:21,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:27:01,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,
1,190725W00118461,b'\x80p\xc4^)\x1bKDKJ\xcb\x98\xa9l\xd6D',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 15:01:32,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,2019-07-24 13:52:21,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 15:01:32,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,
2,190725W00143303,b'\x81J\xa7\x9d-\x92\x9b\x8bL\xa3D_#\x9b|\x8c',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 15:30:38,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,2019-07-24 13:52:21,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 15:30:38,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,
3,190725W00088154,b'\x82\x1b\x8dExG|pBB1\xf6\xc6~\xfes',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:31:52,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,2019-07-24 13:52:21,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:31:52,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,
4,190725W00113236,b'\x82<x\xf5\xb0\xa2x\xd6N\xfd#b\xe1\xa2\xfa%',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:57:49,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,2019-07-24 13:52:21,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:57:49,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Бе

Функция, в которой и происходит тестирование.

In [24]:
@benchmark
def make_test():
    new_df = pd.DataFrame(columns=set(result_df.loc[:, 'Источник']), index=list(result_df.columns.fillna(0)) + ['Всего'])
    nan = float('nan')
    for source in set(result_df['Источник']):
        filtered = result_df[result_df['Источник'] == source]
        new_df.loc['Всего', source] = len(filtered)
        count_s = filtered.replace('', nan).replace('НетДанных', nan).replace('0.000000', nan).count()
        for k, v in count_s.items():
            new_df.loc[k, source] = v * 100 / new_df.loc['Всего', source]
        new_df.loc['КодЗадания', source] = list(filtered['КодЗадания'])[0]
        new_df.loc['КодЗадачи', source] = list(filtered['КодЗадачи'])[0]
    new_df.loc['ДатаЗадания'] = list(filtered['ДатаЗадания'])[0]

    return new_df.T

In [32]:
test = make_test()

make_test выполнилась за время 33.599905252456665s


Выгрузка данных в `csv` файл для удобного просмотра в Excel

In [33]:
# test.to_csv('test_result.csv', header=list(test.columns), encoding='cp1251', sep=';')
test

,Код,Ссылка,КодЗадачи,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,КодЗадания,ДатаЗадания,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Водоснабжение,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Интернет,Класс Объекта,Количество Комнат,Количество Этажей,Кондиционирование,Лифты,Материал Наружных Стен,Минут До Ближайшей Станции Метро,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,Охрана Предложения,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расположение Относительно Первой Линии Домов,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Объекта,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Электричество,Этаж,Всего
Sibdom.Ru,100,100,0001-0405-0024,100,98.8496,0,100,100,100,94.9055,100,99.0961,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0.246508,0.164339,88.0033,80.6902,0.410846,0,0,0,99.0961,99.0961,28.3484,100,100,0,6.1627,0,8.62777,0,0,66.8036,0,0,0,99.0961,0,98.8496,0,0,0,0,0,0,0,0,0.246508,0.0821693,100,97.9458,100,0,0,0,100,0,0,100,0,0,0,0.493016,0,100,97.6993,0,79.7042,1217
Квадрат.ру,100,100,0001-0405-0058,100,99.8,0,100,100,100,98,100,99.8,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0,0,96.4,38.4,1.2,0,0,4.8,0,0,0,100,100,0,1.2,0,91.4,0,0,0,0,16.2,0,99.8,0,99.6,0,0,0,0,0,0,0,0,0,0.2,100,99.6,100,0,0,0,100,0,0,100,0,0,0,1.8,0,100,99.4,0,91.2,500
Rosrealt.ru,100,100,0001-0405-0014,100,95.0964,0,100,100,100,90.456,100,99.0225,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0.214834,0.171867,71.6204,41.6241,8.2389,0,0,0,0,0,0,100,100,0,2.71228,0,20.6348,0,0,0,0,0,0,99.4361,0,99.7476,0,0,0,0,0,0,0,0,0.225576,0.0429669,99.9678,99.4898,100,0,0,0,100,0,0,100,0,0,0,0.843225,0,99.9409,94.8118,0,45.1367,18619
Zdanie.info,100,100,0001-0405-0027,100,99.9865,0,100,100,100,99.314,100,99.9865,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0.295938,0.269034,98.332,79.6341,0.201776,0,0,0,0,0,0,100,100,0,9.48345,0,25.1278,0,0,0,0,0,0,99.9865,0,98.6145,0,0,0,0,0,0,0,0,0.295938,0.094162,100,96.0721,100,0,0,0,100,0,0,100,0,0,0,12.6446,0,84.9206,83.7638,0,29.4861,7434
Адвекс - АН,100,100,0001-0405-0022,100,100,0,100,100,100,100,100,100,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0,0,100,94.7368,0,0,94.7368,0,100,100,0,100,100,0,0,0,63.1579,0,0,0,0,0,0,100,89.4737,100,0,89.4737,0,0,0,0,0,0,0,0,100,100,100,0,0,0,100,0,0,100,94.7368,0,0,0,0,100,100,94.7368,47.3684,19
INFOLINE,100,100,0001-0405-0064,100,100,0,100,100,100,86.5,100,99.5,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0.5,0.5,94.5,84,1,0,0,0,87,87,0,100,100,0,1.5,0,83.5,0,0,0,0,0,0,99.5,0,95,0,0,0,0,0,0,0,0,0.5,0,100,88,100,0,0,0,100,0,0,100,0,0,0,2.5,0,100,95,0,86,200
Доска.ру,100,100,0001-0405-0068,100,92,100,100,100,100,84,100,84,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0,0,84,80,0,0,0,0,0,0,0,100,100,0,8,0,36,0,0,0,0,0,0,84,0,84,0,0,0,0,0,0,0,0,0,0,100,76,100,0,0,0,100,0,0,100,0,24,0,24,0,100,80,0,36,25
Buy business portal,100,100,0001-0405-0070,100,98.6425,100,100,100,100,86.2745,100,99.5475,100,99.6983,100,100,100,0001-0405,2019-07-24 13:52:21,0.15083,0,80.9955,53.092,1.80995,0,0,0,0,0,2.5641,100,100,0,1.5083,0,19.3062,0,0,0,0,0,0,99.5475,0,98.3409,0,0,0,0,0,0,0,0,0.452489,0,100,99.3967,100,0,0,0,100,0,0,100,0,1.05581,0,4.67572,0,99.6983,96.6817,0,25.0377,663
Яндекс Недвижимость,100,100,0001-0405-0071,100,99.7313,100,100,100,100,94.6507,100,99.7802,100,100,100,100,100,0001-0405,2019-07-24 13:52:21,0.

In [34]:
test.loc['Sibdom.Ru'].to_dict()

{'Код': 100.0,
 'Ссылка': 100.0,
 'КодЗадачи': '0001-0405-0024',
 'Источник': 100.0,
 'АдресAhunter': 98.84963023829089,
 'АктуальнаяСсылкаИсточника': 0.0,
 'ДатаПересмотраЭкспозиции': 100.0,
 'ДатаПроверкиАктуальности': 100.0,
 'ДатаРазмещения': 100.0,
 'Город': 94.90550534100247,
 'Описание': 100.0,
 'Подсегмент': 99.0961380443714,
 'Сегмент': 100.0,
 'Субъект': 100.0,
 'СсылкаИсточника': 100.0,
 'ТипРынка': 100.0,
 'ТипСделки': 100.0,
 'КодЗадания': '0001-0405',
 'ДатаЗадания': '2019-07-24 13:52:21',
 '3.Муниципальное Образование': 0.2465078060805259,
 '5.Внутригородской Район': 0.16433853738701726,
 '7.Улица': 88.00328677074774,
 '8.Дом': 80.69022185702548,
 'Адрес (временная)': 0.4108463434675431,
 'Ближайшая Станция Метро': 0.0,
 'Водоснабжение': 0.0,
 'Высота Потолков': 0.0,
 'ГеоКод Долгота': 99.0961380443714,
 'ГеоКод Широта': 99.0961380443714,
 'Год Постройки': 28.348397699260477,
 'Дата Сбора Информации': 100.0,
 'Заголовок': 100.0,
 'Интернет': 0.0,
 'Класс Объекта': 6.1626

## Для тестов


In [ ]:
!pip install pymongo

In [ ]:
cursor = conn.cursor()
query = '''
SELECT TOP 100
    COALESCE(
         CAST(t.Значение_Дата AS varchar)
        ,CAST(t.Значение_Число AS varchar)
        ,t.Значение_Строка
        ,t_dop.Наименование
        ,t_types.Наименование
        ,t_types_1.Значение
        ,t_logic.Значение
        ,t_verif.Значение
        ,t_podseg.Наименование
        ,t_zastr.Наименование
        ,t_stations.Наименование
    ) AS Значение

FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
    LEFT JOIN [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop 
        ON t_dop.Ссылка = t.Значение
    LEFT JOIN [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types
        ON t_types.Ссылка = t.Значение
    LEFT JOIN [rway].[Перечисление].[ТипыСделки] t_types_1 
        ON t_types_1.Ссылка = t.Значение
    LEFT JOIN [rway].[Перечисление].[ЛогическиеЗначения] t_logic 
        ON t_logic.Ссылка = t.Значение
    LEFT JOIN [rway].[Перечисление].[ТипыВерифицированности] t_verif 
        ON t_verif.Ссылка = t.Значение
    LEFT JOIN [rway].[Справочник].[ПодСегменты] t_podseg 
        ON t_podseg.Ссылка = t.Значение
    LEFT JOIN [rway].[Справочник].[Застройщики] t_zastr 
        ON t_zastr.Ссылка = t.Значение
    LEFT JOIN [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations 
        ON t_stations.Ссылка = t.Значение  
    
    
'''
# WHERE _FLD198 = '0001-0402-0001'

cursor.execute(query)
data = cursor.fetchall()
df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
df.columns = list(map(lambda x: x[0], cursor.description))
df


In [ ]:
cursor.execute("SELECT * FROM [rway].[Перечисление].[ТипыСделки] t_types_1 ").fetchone()